# Duking and Punching

### Excersise AR game. Puch the face that you seeon the screen. Harder you hits give you more points. 
### When something is flying at you duck under it or away from it.
### You have 60 seconds
### after that press q to quit or r to restart and play again


In [ ]:

### Instalation notes

### download and install python
###https://www.python.org/downloads/macos/
### install opencv and other requirements
#!pip3 install opencv-python
#!pip3 install mediapipe
#!pip3 install pygame
# !pip install screeninfo
###!pip install pygame

In [1]:

print ("loading game...")

loading game...


In [3]:
### import dependencies
import cv2
import numpy as np
import time
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
try:
    from win32api import GetSystemMetrics
except:
    pass
try:
    from AppKit import NSScreen
except:
    pass
import pygame



In [8]:
### read files used in the game 
face_main = cv2.imread("image/redface.jpg", cv2.IMREAD_COLOR)
p2 = cv2.imread("image/p2.jpg", cv2.IMREAD_COLOR)
p3 = cv2.imread("image/p3.jpg", cv2.IMREAD_COLOR)
p4 = cv2.imread("image/p4.jpg", cv2.IMREAD_COLOR)
p5 = cv2.imread("image/p5.jpg", cv2.IMREAD_COLOR)

glove_red_main = cv2.imread("image/glove_top.jpg", cv2.IMREAD_COLOR)
glove_red_left = cv2.imread("image/glove_left.jpg", cv2.IMREAD_COLOR)
glove_red_right = cv2.imread("image/glove_right.jpg", cv2.IMREAD_COLOR)
glove_red_right = cv2.imread("image/glove_right.jpg", cv2.IMREAD_COLOR)
end_main = cv2.imread("image/end.jpg", cv2.IMREAD_COLOR)

pygame.init()
pygame.mixer.music.load('image/pow.wav')
####https://realpython.com/playing-and-recording-sound-python/
filename = 'image/pow.wav'

In [10]:
### function based on https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html
def image_add(img1,img2,y,x): 
    
    # I want to put logo on top-left corner, So I create a ROI
    rows,cols,channels = img2.shape
    x=int(x-rows/2)
    y=int(y-cols/2)
    roi = img1[x:rows+x, y:cols+y]
    # Now create a mask of logo and create its inverse mask also
    img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)
    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
    # Put logo in ROI and modify the main image
    dst = cv2.add(img1_bg,img2_fg)
    img1[x:rows+x, y:cols+y ] = dst
    return img1

In [12]:
####https://stackoverflow.com/questions/69240807/how-to-change-colors-of-the-tracking-points-and-connector-lines-on-the-output-vi

dot=(min(max(np.random.rand(1)[0],.2),.8),min(max(np.random.rand(1)[0],.3),.7))
score=0        
duck_time=0
face_time=0
duck_random= np.random.rand(1)[0]
sensativity=.1
speed=10
state="nothing"
#percent by which the image is resized
scale_percent = .99
pow_play=0
n=1
hand_previous=[0,0]
hand_left_previous=[0,0]
last_punch_speed=0
t0 = time.time()
g=0
#calculate the 50 percent of original dimensions
try:
    width = int(GetSystemMetrics(0)*scale_percent )
    height =  int(GetSystemMetrics(1)*scale_percent )
except:
    width = int(NSScreen.mainScreen().frame().size.width*scale_percent )
    height =  int(NSScreen.mainScreen().frame().size.height*scale_percent )
    
dsize = (width, height)
end_pic=cv2.resize(end_main, (int(width*.99),int(height*.4)))

# For webcam input:

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) #captureDevice = camera

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    ##
        ###pass
        
        start=time.time()
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue
        try:
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            # Draw the pose annotation on the image.
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image,
                                      results.pose_landmarks, 
                                      mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(0,230,230), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(230,0,0), thickness=2, circle_radius=2)
                                     
                                     
                                     
                                     
                                     )

#### Coding in movement: Calculating distance
            point=mp_pose.PoseLandmark.RIGHT_INDEX.value
            point_l=mp_pose.PoseLandmark.LEFT_INDEX.value
            point_eye=mp_pose.PoseLandmark.LEFT_EYE.value

            visibility =results.pose_landmarks.landmark[point].visibility
            visibility_left =results.pose_landmarks.landmark[point_l].visibility

## Hand positions
            x=((results.pose_landmarks.landmark[point].x))
            y=((results.pose_landmarks.landmark[point].y))
            hand =([x,y])
            speed= int(5*abs(np.sqrt((hand[0]-hand_previous[0])*(hand[0]-hand_previous[0])+(hand[1]-hand_previous[1])*(hand[1]-hand_previous[1]))))
            hand_previous =hand
            xl=((results.pose_landmarks.landmark[point_l].x))
            yl=((results.pose_landmarks.landmark[point_l].y))
            xeye=((results.pose_landmarks.landmark[point_eye].x))
            yeye=((results.pose_landmarks.landmark[point_eye].y))
            hand_left =([xl,yl])
            speed_left= int(10*abs(np.sqrt((hand[0]-hand_previous[0])*(hand[0]-hand_previous[0])+(hand[1]-hand_previous[1])*(hand[1]-hand_previous[1]))))
            hand_left_previous =hand_left
            speed=max(speed,speed_left)

###Distance calculation
            distr= np.sqrt((dot[0]-x)*(dot[0]-x)+(dot[1]-y)*(dot[1]-y))
            distl= np.sqrt((dot[0]-xl)*(dot[0]-xl)+(dot[1]-yl)*(dot[1]-yl))
            dist=min(distr,distl)
        except:
            pass
        
        t1 = time.time()
        timepass=t1-t0
        
        
        #### Coding in movement: making movement 
        image=cv2.resize(image, dsize)
        rows,cols,channels = image.shape
        ##end=cv2.resize(end, (int(cols/2),int(rows/6)))
        
        
        if timepass<60:
             # Flip the image horizontally for a selfie-view display.
                
##### Changin the face 
            face=cv2.resize(face_main, (300,300))
            if timepass>30 and score>20:
                face=cv2.resize(p3, (300,300))
            if timepass>40 and score>30:
                face=cv2.resize(p4, (300,300))
            if timepass>55 and score>30:
                face=cv2.resize(p5, (300,300))
#### Ducking instructions
            duck_frequency=5
            
            
            
            if n%duck_frequency==0:
                
                ###glove_red =cv2.resize(glove_red , (100+duck_time*10,100+duck_time*10))
                if duck_random<.3:
                    ### midle punch
                    ##cv2.putText(image, "DUCK"+str(duck_time),tuple([int(rows*.5), int(cols*.3)]), cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 255, 255), 3, cv2.LINE_AA)
                    glove_red_main =cv2.resize(glove_red_main  , (200+min(200,duck_time*7),200+min(200,duck_time*7)))
                    image=image_add(image,glove_red_main,int(width*.5), int(height*min(.5,.1+.01*duck_time)))  
                    ##glove_red
                    duck_time=duck_time+1
                    if duck_time%10==0:
                        score=score-1
                    ##time.sleep(10)
                    if yeye>.5:
                        n=n+1 
                        duck_time=0
                        duck_random= np.random.rand(1)[0]
                elif duck_random<.6:
                    ### rightt Punch
                    glove_red_left =cv2.resize(glove_red_left  , (200+min(400,duck_time*7),200+min(400,duck_time*7)))
                    ##cv2.putText(image, "right"+str(duck_random),tuple([int(rows*.5), int(cols*.3)]), cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 255, 255), 3, cv2.LINE_AA)
                    image=image_add(image,glove_red_left,int(width*min(.4,.1+.01*duck_time)), int(height*.3))  
                    ##glove_red
                    duck_time=duck_time+1
                    if duck_time%10==0:
                        score=score-1
                    ##time.sleep(10)
                    if xeye>.6 or yeye>.5:
                        n=n+1 
                        duck_time=0
                        duck_random= np.random.rand(1)[0]
                else:
                    #### left punch
                    glove_red_right =cv2.resize(glove_red_right , (200+min(400,duck_time*7),200+min(400,duck_time*7)))
                    ##cv2.putText(image, "left punch"+str(duck_random),tuple([int(rows*.5), int(cols*.3)]), cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 255, 255), 3, cv2.LINE_AA)
                    image=image_add(image,glove_red_right,int(width*max(.6,.9-.01*duck_time)), int(height*.30))  
                    ##glove_red
                    duck_time=duck_time+1
                    if duck_time%10==0:
                        score=score-1
                    ##time.sleep(10)
                    if xeye<.4 or yeye>.5:
                        n=n+1 
                        duck_time=0
                        duck_random= np.random.rand(1)[0]
#### punching instructions
            elif dist>.1 and face_time<50:
                face_time=face_time+1
                ##cv2.putText(image, "IB "+str(speed),tuple([int(cols*dot[0]), int(rows*dot[1])]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
                image=image_add(image,face,int(cols*dot[0]), int(rows*dot[1])) 
            else:
                ### hand meets the face so punch and reset
                ###cv2.putText(image, "BAM"+str(round(dist,2)),tuple([int(cols/2), int(rows/2)]), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 255), 3, cv2.LINE_AA)
                
                dot=(min(max(np.random.rand(1)[0],.2),.8),min(max(np.random.rand(1)[0],.3),.7))
#### if you dont punch fast enough it will move
                if face_time<50:
                    score=score+1+speed
                    last_punch_speed=speed
                    ##cv2.putText(image, "",tuple([int(cols/2), int(rows/2)]), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 255), 3, cv2.LINE_AA)
                    timepass=1
                    pygame.mixer.music.play(1)
                    

                n=n+1
                
                face_time=0
                
            image=cv2.flip(image, 1)
        
           ## cv2.putText(image, "SCORE:"+str(score)+" |Time:"+str(int(timepass))+" | Bonus Force:"+str(int(last_punch_speed)),tuple([10, 100]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
            if score>0:
                ####
                cv2.putText(image, " SCORE:"+str(score)+" |Time:"+str(int(timepass)),tuple([10, 100]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
                cv2.putText(image, " q to quit",tuple([int(0), int(rows-120)]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
            else:
                ####punching instructions
                cv2.putText(image, " SCORE:"+str(score)+" |Time:"+str(int(timepass)),tuple([10, 100]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
                cv2.putText(image, " PUNCH PUTIN IN THE FACE",tuple([100, int(rows/2)]), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 255), 3, cv2.LINE_AA)
            if n==duck_frequency:
                ####duckign instructions
                cv2.putText(image, " DUCK AWAY FROM THE GRANADE",tuple([100, int(rows/2)]), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)

                
            
        else:
            image=cv2.flip(image, 1)
            cv2.putText(image, "FINAL SCORE:"+str(score),tuple([int(cols/6), int(rows/2)]), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 255), 3, cv2.LINE_AA)
            ###cv2.putText(image, str(score),tuple([int(cols/3), int(rows/2)]), cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 255, 255), 3, cv2.LINE_AA)
            image=image_add(image,end_pic,int(cols*.5), int(rows*3/4))
            cv2.putText(image, " r to restart",tuple([int(0), int(rows-120)]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
             
            if cv2.waitKey(1) & 0xFF == ord('r'):
                ###print("reset")
                t0 = time.time()
                score=0
        cv2.imshow('Ducking and Punching', image)


                

        end=time.time()
        totalTime=end-start
        fps=1/totalTime   
        



        if cv2.waitKey(1) & 0xFF == ord('q'):
              break
cap.release()
cv2.destroyAllWindows()
print (fps)

34.96331368838725


528.0